## Task 1: Data and Tokenizer
1. Import the data
2. Create a tokenizer
3. Char to index and Index to char dictionaries


In [ ]:
!git clone https://github.com/am1tyadav/superhero

: 

In [ ]:
with open('superhero/superheroes.txt','r') as f:
    data = f.read()
data[:100]

: 

In [ ]:
import tensorflow as tf

: 

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~',
    split='\n',
)

: 

In [ ]:
tokenizer.fit_on_texts(data)

: 

In [ ]:
char_to_index = tokenizer.word_index
index_to_char = dict((v,k) for k , v in char_to_index.items())
print(index_to_char)

: 

## Task 2: Names and Sequences
1. Converting between names and sequences


In [ ]:
names = data.splitlines()
names[:10]

: 

In [ ]:
tokenizer.texts_to_sequences(names[0])

: 

In [ ]:
def name_to_seq(name):
    return[tokenizer.texts_to_sequences(c)[0][0] for c in name]

: 

In [ ]:
name_to_seq(names[0])

: 

In [ ]:
def seq_to_name(seq):
    return ''.join([index_to_char[i] for i in seq if i != 0])

: 

In [ ]:
seq_to_name(name_to_seq(names[0]))

: 

In [ ]:
print(names[1])
print(tokenizer.texts_to_sequences(names[1]))
print(name_to_seq(names[1]))
print(seq_to_name(name_to_seq(names[1])))

: 

## Task 3: Creating Examples
1. Creating sequences
2. Padding all sequences

In [ ]:
sequences= []
for name in names:
    seq = name_to_seq(name)
    if len(seq) >=2:
        sequences += [seq[:i] for i in  range(2, len(seq) + 1)]

: 

In [ ]:
sequences[5:16]

: 

In [ ]:
max_len = max([len(x) for x in sequences])
print(max_len)

: 

In [ ]:
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(
    sequences, 
    padding='pre',
    maxlen= max_len
)
print(padded_sequences[1])

: 

In [ ]:
padded_sequences.shape

: 

## Task 4: Training and Validation Sets
1. Creating training and validation sets

In [ ]:
x, y = padded_sequences[:, :-1], padded_sequences[:, -1]
print(x.shape, y.shape)

: 

In [ ]:
from sklearn.model_selection import train_test_split

(x_train, x_test), (y_train, y_test) =  train_test_split(x, y)
print(x_test.shape, y_test.shape)
print(x_train.shape, y_train.shape)

: 

In [ ]:
num_chars = len(char_to_index.keys()) + 1
print(num_chars)

: 

## Task 5: Creating the Model


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPool1D, LSTM
from tensorflow.keras.layers import Bidirectional, Dense

: 

In [ ]:
num_chars = 29
model = Sequential([
                    Embedding(num_chars, 8, input_length=max_len-1),
                    Conv1D(64, 5, strides=1, activation='tanh', padding='causal'),
                    MaxPool1D(2),
                    LSTM(32),
                    Dense(num_chars, activation='softmax')
])

model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
 
model.summary()

: 

## Task 6: Training the Model


In [ ]:
h = model.fit(
    x_train, y_train,
    validation_data = (x_test,y_test), 
    epochs = 25, verbose = 2,
    callbacks=[ tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 3)]
)

: 

In [ ]:
from matplotlib import pyplot as plt

epochs_ran = len(h.history['loss'])
plt.plot(range(0,epochs_ran), h.history['val_accuracy'], label = 'Validation')
plt.plot(range(0,epochs_ran), h.history['accuracy'], label = 'Training')
plt.legend()
plt.show()

: 

## Task 7: Generating Names


In [ ]:
def generate_names(seed):
    for i in range(0,40):
        seq = name_to_seq(seed)
        padded = tf.keras.preprocessing.sequence.pad_sequences([seq], padding = 'pre', maxlen = max_len-1,truncating='pre')
        pred = model.predict(padded)[0]
        pred_char = index_to_char[tf.argmax(pred).numpy()]
        seed == pred_char
        if pred_char == '\t':
            break
    print(seed)

: 

In [ ]:
generate_names('a')

: 